In [1]:
pip install pandas hvplot panel

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import panel as pn
import hvplot.pandas

%matplotlib inline

In [3]:
# Load the dataset
df = pd.read_excel('C:/Users/USER/Desktop/FimaProyecto/Horas de producción.xlsx')

In [4]:
idf = pn.interact(lambda: df)
# Eliminar espacios en blanco alrededor de los nombres de las columnas
df.columns = df.columns.str.strip()


In [5]:
# Obtener la fecha mínima y máxima del DataFrame
fecha_min = df['Fecha'].min()
fecha_max = df['Fecha'].max()

# Crear el IntRangeSlider con las fechas mínima y máxima
week_slider = pn.widgets.DateRangeSlider(name='Fechas', start=fecha_min, end=fecha_max)

# Mostrar el widget
week_slider

pn.extension()

In [6]:
y_axis = pn.widgets.RadioButtonGroup(
    name = 'Y axis',
    options=['CALF','CALH','PINS', 'CALL', 'CALL(DB+RL)', 'MEC', 'HABL', 'ENS', 'PINT'],
    button_type='success'    
)

In [7]:

fecha_min = df['Fecha'].min()
fecha_max = df['Fecha'].max()

# Crear el IntRangeSlider con las fechas mínima y máxima
week_slider = pn.widgets.DateRangeSlider(name='Fechas', start=fecha_min, end=fecha_max)

# Obtener los valores seleccionados en el rango de fechas del week_slider
fecha_inicio = pd.to_datetime(week_slider.value[0])
fecha_fin = pd.to_datetime(week_slider.value[1])

# Filtrar el DataFrame por las fechas seleccionadas y las contratas específicas
contratas=['CALF','CALH','PINS','CALL','CALL(DB+RL)','MEC','HABL','ENS','PINT']
contrata_filtered = df[(df['Fecha'] >= fecha_inicio) & (df['Fecha'] <= fecha_fin) & (df['GrupoBal'].isin(contratas))]

# Eliminar columnas no numéricas que no son relevantes para el cálculo de la media
contrata_filtered_numeric = contrata_filtered[['Fecha', 'GrupoBal', 'Total horas Prod', 'Hrs. Muertas Acum']]

# Crear una función para generar el gráfico interactivo
def create_contrata_plot(y_axis):
    # Calcular la media de las columnas numéricas
    contrata_pipeline = contrata_filtered_numeric.groupby(['GrupoBal', 'Fecha']).mean().reset_index()
    
    # Crear el gráfico utilizando hvplot
    contrata_plot = contrata_pipeline.hvplot(x='Fecha', y=y_axis, by='GrupoBal', title='Contrata por fecha')
    
    return contrata_plot

# Crear el widget RadioButtonGroup para seleccionar el eje y
horas_axis = pn.widgets.RadioButtonGroup(
    name='Y axis',
    options=['Total horas Prod', 'Hrs. Muertas Acum'],
    button_type='success'
)

# Crear la aplicación Panel
@pn.depends(horas_axis=horas_axis.param.value)
def actualizar_plot(horas_axis):
    return create_contrata_plot(horas_axis)

# Mostrar la aplicación Panel
grafico = pn.Column(
    pn.Row(horas_axis),
    actualizar_plot
)

grafico.servable()
grafico.show()

Launching server at http://localhost:61122


In [177]:
# import panel as pn
# import pandas as pd
# import hvplot.pandas

# fecha_min = df['Fecha'].min()
# fecha_max = df['Fecha'].max()

# # Crear el IntRangeSlider con las fechas mínima y máxima
# week_slider = pn.widgets.DateRangeSlider(name='Fechas', start=fecha_min, end=fecha_max)

# # Obtener los valores seleccionados en el rango de fechas del week_slider
# fecha_inicio = pd.to_datetime(week_slider.value[0])
# fecha_fin = pd.to_datetime(week_slider.value[1])

# # Filtrar el DataFrame por las fechas seleccionadas y las contratas específicas
# contratas=['CALF','CALH','PINS','CALL','CALL(DB+RL)','MEC','HABL','ENS','PINT']
# contrata_filtered = df[(df['Fecha'] >= fecha_inicio) & (df['Fecha'] <= fecha_fin) & (df['GrupoBal'].isin(contratas))]

# # Eliminar columnas no numéricas que no son relevantes para el cálculo de la media
# contrata_filtered_numeric = contrata_filtered[['Fecha', 'GrupoBal', 'Total horas Prod', 'Hrs. Muertas Acum']]

# # Crear una función para generar el gráfico interactivo
# def create_contrata_plot(y_axis):
#     # Calcular la media de las columnas numéricas
#     contrata_pipeline = contrata_filtered_numeric.groupby(['GrupoBal', 'Fecha']).mean().reset_index()
    
#     # Crear el gráfico utilizando hvplot
#     contrata_plot = contrata_pipeline.hvplot(x='Fecha', y=y_axis, by='GrupoBal', title='Horas Producidas y Muertas por cada subcontrata', height=300, width=700)
    
#     return contrata_plot

# # Crear el widget RadioButtonGroup para seleccionar el eje y
# horas_axis = pn.widgets.RadioButtonGroup(
#     name='Y axis',
#     options=['Total horas Prod', 'Hrs. Muertas Acum'],
#     button_type='success'
# )

# # Crear la aplicación Panel
# @pn.depends(horas_axis=horas_axis.param.value)
# def update_plot(horas_axis):
#     return create_contrata_plot(horas_axis)

# # Mostrar la aplicación Panel
# grafico = pn.Column(
#     pn.Row(horas_axis),
#     update_plot
# )

# grafico.servable()
# grafico.show()


Launching server at http://localhost:61022


In [8]:

# Obtener los datos para el gráfico de pastel inicial
grouped_df = df.groupby('GrupoBal')['Total horas Prod'].sum().reset_index()
fig = px.pie(grouped_df, values='Total horas Prod', names='GrupoBal',
             title='Total de Horas de Producción')



def update_pie_chart(week):
    # Filtrar el DataFrame por la semana seleccionada
    filtered_df = df[df['Semana'] == week]
    
    # Agrupar los datos filtrados por GrupoBal y sumar las horas producidas
    grouped_df = filtered_df.groupby('GrupoBal')['Total horas Prod'].sum().reset_index()
    
    # Actualizar el gráfico de pastel con los datos filtrados
    fig.data[0].labels = grouped_df['GrupoBal']
    fig.data[0].values = grouped_df['Total horas Prod']
    fig.update_layout(title=f'Total de Horas de Producción - Semana {week}')

# Definir la función de actualización del gráfico cuando se cambia la semana
week_slider.param.watch(update_pie_chart, 'value')

# Mostrar el layout con el widget del selector de semana y el gráfico de pastel
pie = pn.Column(
    fig,
    sizing_mode="stretch_both"
)

pie.servable()
pie.show()

Launching server at http://localhost:61129


In [9]:


# Obtener la lista de contratas disponibles
contratas = df['GrupoBal'].unique()

# Crear el widget de selección de contrata
contrata_radio_buttons = pn.widgets.RadioButtonGroup(name='Seleccionar Contrata', options=contratas.tolist())

# Crear la figura inicial con la primera contrata
initial_contrata_df = df[df['GrupoBal'] == contratas[0]]
fig = px.scatter(initial_contrata_df, x='Hrs. Muertas Acum', y='Total horas Prod', color='Machine', 
                 color_discrete_sequence=px.colors.qualitative.Dark24,
                 labels={'Total horas Prod': 'Horas Producidas Totales', 'Hrs. Muertas Acum': 'Horas Muertas'})
fig.update_layout(title=f'Horas Muertas vs. Horas Producidas Totales para Contrata {contratas[0]}',
                  xaxis_title='Horas Muertas',
                  yaxis_title='Horas Producidas Totales', height=300, width=700)

# Crear un Panel para el gráfico
plot_panel = pn.panel(fig)


def update_ploteo(event):
    contrata = event.new

    # Filtrar el DataFrame solo para la contrata seleccionada
    contrata_df = df[df['GrupoBal'] == contrata]

    # Crear la figura actualizada para la contrata seleccionada
    fig = px.scatter(contrata_df, x='Hrs. Muertas Acum', y='Total horas Prod', color='Machine', 
                     color_discrete_sequence=px.colors.qualitative.Dark24,
                     labels={'Total horas Prod': 'Horas Producidas Totales', 'Hrs. Muertas Acum': 'Horas Muertas'})
    fig.update_layout(title=f'Horas Muertas vs. Horas Producidas Totales para Contrata {contrata}',
                      xaxis_title='Horas Muertas',
                      yaxis_title='Horas Producidas Totales', height=300, width=700)

    # Mostrar el gráfico actualizado
    plot_panel.object = fig


# Definir la función de actualización del gráfico cuando se cambia la selección de la contrata
contrata_radio_buttons.param.watch(update_ploteo, 'value')

# Crear un layout de Panel con el widget de selección de contrata y el gráfico
layout = pn.Column(
    contrata_radio_buttons,
    plot_panel
)

# Mostrar el layout
layout.servable()
layout.show()

Launching server at http://localhost:61140


In [21]:
template = pn.template.FastListTemplate(
    title = 'Dashboard de seguimiento semanal',
    sidebar = [pn.pane.Markdown('### Seguimiento semanal de horas'),
               pn.pane.Markdown('El objetivo de este dashboard es obtener insights a partir de las horas de producción diarias a lo largo de la semana, y la relación entre horas de producción diaria y horas muertas diarias con respecto a las máquinas de las diferentes subcontratas.'),
               pn.pane.PNG('C:/Users/USER/Desktop/FimaProyecto/fima.png'),
               ],
    main = [pn.Row(pn.Column(contrata_radio_buttons,plot_panel)), 
            pn.Row(pn.Column(pn.Row(horas_axis),actualizar_plot))],
    accent_base_color="#88d8b0",
            #pn.Row(pn.Column(horas_axis,update_plot))]
)



template.servable()
template.show()

Launching server at http://localhost:61314


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358 entries, 0 to 357
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Cod Empl.               358 non-null    int64         
 1   Fecha                   358 non-null    datetime64[ns]
 2   Empleado                358 non-null    object        
 3   Turno                   358 non-null    int64         
 4   WO                      358 non-null    object        
 5   Suffix                  358 non-null    int64         
 6   Seq                     358 non-null    int64         
 7   GrupoBal                358 non-null    object        
 8   Machine                 358 non-null    object        
 9   Total horas Prod        358 non-null    float64       
 10  Hrs.Indirectas          358 non-null    float64       
 11  Hrs. Directas Int       358 non-null    int64         
 12  Hrs. Directas           358 non-null    float64   